In [1]:
# 폴더별 확진자 동선 공개파일을 기준일 당일 데이터만 합침

In [1]:
import pandas as pd
import numpy as np
import os
import re
import time

from selenium import webdriver
import selenium.webdriver.common.keys as Keys
from bs4 import BeautifulSoup

In [18]:
# initial_path 아래 있는 원하는 형식의 파일명만 골라서 리스트에 담아 반환 (재귀함수)
def check_all_files_underpath(initial_path, file_name) :
    dir_list = os.listdir(initial_path)
    direc_list = [i for i in dir_list if os.path.isdir(initial_path+'/'+i)]
    
    accum_file_list = []
    
    if len(direc_list) :
        for i in direc_list :
            print(initial_path+'/'+i, '발견! ')
            temp_list = check_all_files_underpath(initial_path+'/'+i, file_name)


            temp_list = [(initial_path.split('/')[-1] + '/'+i) for i in temp_list]

            
            accum_file_list = accum_file_list + temp_list
            
        return accum_file_list
    else : 
        print('최하위 디렉토리 도착!')
#         print(os.listdir(initial_path))
        
        temp_list = os.listdir(initial_path)
        temp_list = [(initial_path.split('/')[-1] + '/'+i) for i in temp_list if re.search(file_name,i) is not None]
        return temp_list
        
    return temp_list

In [19]:
# 확진자의 돈성 파일목록을 받아와 하나씩 읽어서 이어 붙이는 클래스
class make_merged_dataframe :
    
    def __init__(self, filelist) :  
        self.filelist = filelist
        self.filelist = ['./'+i for i in self.filelist]
    
    # 데이터를 합치기 위한 변수 'df99'가 이미 생성 되어 있는지 확인
    def check_var (self) :
        return ('df99' in globals())
    
    # 파일의 이름에서 기준월,일을 뽑아 내기 위한 함수
    def check_date_meta(self, filename) :
        m = re.search(r'\d[.]\d\d', filename)
        print(m.group().split('.')[0],'월',m.group().split('.')[1],'일 데이터')
        return((m.group().split('.')[0],m.group().split('.')[1]))
    
    # 새로운 데이터 프레임 생성
    def create_new_dataframe(self, dataframe) :
        global df99
        df99 = dataframe
    
    # 'df99'가 있을 경우에 데이터 들을 합치는 함수
    def merge_exist_dataframe(self, dataframe) :
        global df99
        df99 = pd.concat([df99,dataframe])
    
    # 파일을 읽어서 데이터프레임으로 반환
    def open_csv(self, path, month, date) :
        df = pd.read_csv(path)
        
        if self.check_var() :
            self.merge_exist_dataframe(df)
            print('"df99"발견. 기존 데이터 프레임에 합체')
        else :
            self.create_new_dataframe(df)
            print('"df99" 없음. 새로 생성')   
    # 클래스의 메인 함수   
    def start(self) :
        for i in self.filelist :
            temp = i.split('/')[-1]
            (m,d) = self.check_date_meta(temp)
            self.open_csv(i,m,d)
            
    
            
        


In [22]:
# 크롤링을 통해 확진자 방문지의 좌표에서 주소를 찾아냄
class check_address :
    def __init__ (self) :
        driver = webdriver.Chrome()
        addr_dict={}
        self.url = 'http://map.kakao.com'
        self.driver = driver
        self.driver.get(self.url)
        self.addr_dict=addr_dict
    
    # 좌표를 받아 다음지도에서 검색해 시/도, 구/시, 동/구 를 크롤링 하는 함수
    def geocode_crawling(self, geo_code) :
        self.driver.find_element_by_xpath('//*[@id="search.keyword.query"]').send_keys(geo_code+'\n')
        time.sleep(0.5)
        html = self.driver.page_source
        page_data = BeautifulSoup(html, 'html.parser')
        try :
            self.driver.find_element_by_xpath('//*[@id="view.mapContainer"]/div[2]/div/div[6]/div[2]/div/div[2]/div[1]')
        except :
            prov = county = town = None
            self.driver.find_element_by_xpath('//*[@id="search.keyword.query"]').clear()
            return (prov, county, town)
            
        try : 
            prov = page_data.select('div.province')[0].text.strip()
        except :
            print('prov 에러! : ',prov)
            prov = None
            
        try :
            county = page_data.select('div.county')[0].text.strip()
        except :
            print('county 에러! : ',prov)
            county = None
            
        try :
            town = page_data.select('div.town')[0].text.strip()
        except :
            print('county 에러! : ',prov)
            town = None
        self.driver.find_element_by_xpath('//*[@id="search.keyword.query"]').clear()
        return (prov, county, town)
    
    # 크롤링 시간을 단축하기 위해 중복되는 좌표들을 제거한 뒤 크롤링 해 다시 원상태로 중복 데이터들까지 살려줌
    def un_unique_geo(self) :
        df99['시'] = np.nan
        df99['구'] = np.nan
        df99['동'] = np.nan
        
        colum_num = list(df99.columns).index('좌표')
        si_colum_num = list(df99.columns).index('시')
        gu_colum_num = list(df99.columns).index('구')
        dong_colum_num = list(df99.columns).index('동')
        
        for i in range(len(df99)) :
            try :
                float(df99.iloc[i,colum_num].split(',')[0])
                float(df99.iloc[i,colum_num].split(',')[1])
                
            except :
                continue
                
            (si,gu,dong) = self.addr_dict[df99.iloc[i,colum_num]]
            
            df99.iloc[i,si_colum_num] = si
            df99.iloc[i,gu_colum_num] = gu
            df99.iloc[i,dong_colum_num] = dong
            
        
    # 클래스의 메인 함수, 좌표 리스트에서 중복을 제거한 unique 값으로 크롤링 시작
    def start_geocode(self, geo_list) :
        addr = []
        geo_unique_df = pd.DataFrame(geo_list.unique())
        
        cnt=0
        for i in geo_unique_df.loc[:,0] :
            print(i,'검색시작 [',cnt,'/',len(geo_unique_df),']')
            cnt += 1
            try :
                float(i.split(',')[0])
                float(i.split(',')[1])
            except :
                print(i,':',cnt,'번 데이터 형식 오류')
                continue
            prov, county, town = self.geocode_crawling(i)
            self.addr_dict[i] = ([prov, county, town])

        self.un_unique_geo()
        self.driver.close()
        del cnt 
        
    

In [23]:
if 'df99' in globals():
    del df99

path = './infection_data'

file_list = check_all_files_underpath(path, '확진자이동')

data = make_merged_dataframe(file_list)
data.start()

addr = check_address()

addr.start_geocode(df99['좌표'])


df99.to_csv('infection_data.csv')



./infection_data/20200330 발견! 
./infection_data/20200330/20200330 발견! 
최하위 디렉토리 도착!
./infection_data/20200331 발견! 
./infection_data/20200331/20200331 발견! 
최하위 디렉토리 도착!
./infection_data/20200401 발견! 
./infection_data/20200401/20200401 발견! 
최하위 디렉토리 도착!
./infection_data/20200402 발견! 
./infection_data/20200402/20200402 발견! 
최하위 디렉토리 도착!
./infection_data/20200402-2 발견! 
./infection_data/20200402-2/20200402 발견! 
최하위 디렉토리 도착!
./infection_data/20200403 발견! 
./infection_data/20200403/20200403 발견! 
최하위 디렉토리 도착!
./infection_data/20200403/__MACOSX 발견! 
./infection_data/20200403/__MACOSX/20200403 발견! 
최하위 디렉토리 도착!
./infection_data/20200405 발견! 
./infection_data/20200405/20200405 발견! 
최하위 디렉토리 도착!
./infection_data/20200406 발견! 
./infection_data/20200406/20200406 발견! 
최하위 디렉토리 도착!
./infection_data/20200407 발견! 
./infection_data/20200407/20200407 발견! 
최하위 디렉토리 도착!
./infection_data/20200408 발견! 
./infection_data/20200408/20200408 발견! 
최하위 디렉토리 도착!
./infection_data/20200409 발견! 
최하위 디렉토리 도착!
./infectio

37.46930050802611,126.936171202131 검색시작 [ 46 / 3359 ]
37.4712548919929,126.93803706490098 검색시작 [ 47 / 3359 ]
37.4802114757342,126.952886855414 검색시작 [ 48 / 3359 ]
26),ddress_me경남창녕군창녕읍교상리40 검색시작 [ 49 / 3359 ]
26),ddress_me경남창녕군창녕읍교상리40 : 50 번 데이터 형식 오류
37.4717890189629,126.939442929767 검색시작 [ 50 / 3359 ]
35.54132017412541,128.49975480303215 검색시작 [ 51 / 3359 ]
ddress_me경남창녕군창녕읍교리288-6,ddress_eishBccoffeeshop(2/23 검색시작 [ 52 / 3359 ]
ddress_me경남창녕군창녕읍교리288-6,ddress_eishBccoffeeshop(2/23 : 53 번 데이터 형식 오류
37.508908482648,126.89131250085099 검색시작 [ 53 / 3359 ]
35.537532831031704,128.49500498547116 검색시작 [ 54 / 3359 ]
ddress_eishShiCh-yeoIerMedicieCiic,35.543200534162 검색시작 [ 55 / 3359 ]
ddress_eishShiCh-yeoIerMedicieCiic,35.543200534162 : 56 번 데이터 형식 오류
37.50493747996278,126.89703983602185 검색시작 [ 56 / 3359 ]
35.5430095338344,128.49317921943614 검색시작 [ 57 / 3359 ]
ddress_eishTimesSqure-B28Secods,B1BskiRobbis 검색시작 [ 58 / 3359 ]
ddress_eishTimesSqure-B28Secods,B1BskiRobbis : 59 번 데이터 형식 오류
37.508138

37.578819585332106,127.055839069304 검색시작 [ 183 / 3359 ]
37.574634998352714,127.038774081903 검색시작 [ 184 / 3359 ]
37.592123503232095,127.052216625932 검색시작 [ 185 / 3359 ]
37.604234501517695,126.90441935029999 검색시작 [ 186 / 3359 ]
35.3179455191433,128.58527723747721 검색시작 [ 187 / 3359 ]
35.3179452884953,128.5852948259692 검색시작 [ 188 / 3359 ]
35.31793000513752,128.58453582985078 검색시작 [ 189 / 3359 ]
35.31783370446071,128.58418209142613 검색시작 [ 190 / 3359 ]
35.2310359687634,128.60841223558327 검색시작 [ 191 / 3359 ]
35.23716112660521,128.64571501966776 검색시작 [ 192 / 3359 ]
35.323281,128.588369 검색시작 [ 193 / 3359 ]
35.305434,128.596415 검색시작 [ 194 / 3359 ]
ddress_eishEderyCreCommuiyChess(2/24~2/29,Sef-Subscripio) 검색시작 [ 195 / 3359 ]
ddress_eishEderyCreCommuiyChess(2/24~2/29,Sef-Subscripio) : 196 번 데이터 형식 오류
35.8131478948436,128.73252834516444 검색시작 [ 196 / 3359 ]
35.813076145831886,128.7325046595384 검색시작 [ 197 / 3359 ]
ddress_me경북경산시중방동516-4,ddress_eishGyeobukUiedIerMedicieIsiue(visi2/21 검색시작 [ 198 / 3359

37.2222144280217,127.05748041028887 검색시작 [ 308 / 3359 ]
37.1487243508812,127.12873766181102 검색시작 [ 309 / 3359 ]
37.22295822780649,127.058433075475 검색시작 [ 310 / 3359 ]
37.20701972779511,127.07842533784334 검색시작 [ 311 / 3359 ]
37.2408545560098,127.05523264737499 검색시작 [ 312 / 3359 ]
37.2146251012973,127.04195542666999 검색시작 [ 313 / 3359 ]
37.21325235123861,127.040665833668 검색시작 [ 314 / 3359 ]
37.215345210172494,127.043970245733 검색시작 [ 315 / 3359 ]
37.21540648654431,127.04395676168498 검색시작 [ 316 / 3359 ]
37.21519971242149,127.04267904203891 검색시작 [ 317 / 3359 ]
37.20701976581681,127.07836788590825 검색시작 [ 318 / 3359 ]
37.01735255326614,127.26036636531445 검색시작 [ 319 / 3359 ]
37.19873692102231,127.053066318613 검색시작 [ 320 / 3359 ]
37.754896406370285,126.77962654033324 검색시작 [ 321 / 3359 ]
36.826119314304215,127.95500850208751 검색시작 [ 322 / 3359 ]
36.823813336624724,127.95491259417248 검색시작 [ 323 / 3359 ]
36.80803074589061,127.79662008127885 검색시작 [ 324 / 3359 ]
36.8307224627675,127.12759752824 검색시작 [

37.35894935805871,126.947724109719 검색시작 [ 434 / 3359 ]
37.3722140219958,126.93619936197 검색시작 [ 435 / 3359 ]
35.80461354373736,127.91798357524209 검색시작 [ 436 / 3359 ]
3/4,6) 검색시작 [ 437 / 3359 ]
3/4,6) : 438 번 데이터 형식 오류
35.84828227314401,129.2175610533782 검색시작 [ 438 / 3359 ]
35.858400575416816,129.19669754214908 검색시작 [ 439 / 3359 ]
35.21461323832531,128.12291170718302 검색시작 [ 440 / 3359 ]
35.1499932065718,129.00392878106715 검색시작 [ 441 / 3359 ]
ddress_eishsiecirce,35.154445174222374 검색시작 [ 442 / 3359 ]
ddress_eishsiecirce,35.154445174222374 : 443 번 데이터 형식 오류
ddress_eishNewSePhrmcy,35.15447320853411 검색시작 [ 443 / 3359 ]
ddress_eishNewSePhrmcy,35.15447320853411 : 444 번 데이터 형식 오류
35.15657214945681,128.99561023061415 검색시작 [ 444 / 3359 ]
35.150005804851595,129.00403881111512 검색시작 [ 445 / 3359 ]
35.14999750954781,129.00410444027915 검색시작 [ 446 / 3359 ]
35.1505429162274,129.00817313363916 검색시작 [ 447 / 3359 ]
35.2058905832124,129.12175452060868 검색시작 [ 448 / 3359 ]
35.20598163419874,129.12175249834036

...

37.48700150141791,126.98275003718702 검색시작 [ 3165 / 3359 ]
36.36621343043236,127.43932132474806 검색시작 [ 3166 / 3359 ]
36.374671010584095,127.425601835097 검색시작 [ 3167 / 3359 ]
36.36623372278159,127.439194413308 검색시작 [ 3168 / 3359 ]
36.36623732738944,127.43919443356612 검색시작 [ 3169 / 3359 ]
36.32765975679301,127.40557098491799 검색시작 [ 3170 / 3359 ]
36.32759128361771,127.40556617522502 검색시작 [ 3171 / 3359 ]
36.3890222508994,127.309254753802 검색시작 [ 3172 / 3359 ]
35.820194504185494,127.14487880394 검색시작 [ 3173 / 3359 ]
35.819612052632344,127.14507582314234 검색시작 [ 3174 / 3359 ]
37.51469694949661,127.119450556946 검색시작 [ 3175 / 3359 ]
37.49965341013924,127.10463906602844 검색시작 [ 3176 / 3359 ]
37.50281724753171,127.096763103578 검색시작 [ 3177 / 3359 ]
36.2962130386256,127.380074702497 검색시작 [ 3178 / 3359 ]
36.32222356176526,127.41040828054733 검색시작 [ 3179 / 3359 ]
36.36275991438219,127.29586937109401 검색시작 [ 3180 / 3359 ]
36.3655881150026,127.25253998879701 검색시작 [ 3181 / 3359 ]
36.34011009344269,127.3938104

36.3586552123159,127.34457028188513 검색시작 [ 3307 / 3359 ]
36.3579585684786,127.346763190351 검색시작 [ 3308 / 3359 ]
36.35877045708289,127.343981401194 검색시작 [ 3309 / 3359 ]
36.3588139633455,127.34514477245 검색시작 [ 3310 / 3359 ]
36.35738936949271,127.346644795305 검색시작 [ 3311 / 3359 ]
36.354090326502615,127.377806832844 검색시작 [ 3312 / 3359 ]
36.36254141191681,127.357341773295 검색시작 [ 3313 / 3359 ]
36.31267852821921,127.380359492734 검색시작 [ 3314 / 3359 ]
36.351955207607,127.378166383103 검색시작 [ 3315 / 3359 ]
36.3546145554774,127.336271388304 검색시작 [ 3316 / 3359 ]
36.3524465539644,127.338106957987 검색시작 [ 3317 / 3359 ]
36.815021888227314,127.11488694693728 검색시작 [ 3318 / 3359 ]
37.44858316606789,127.157543441952 검색시작 [ 3319 / 3359 ]
37.443614611720314,126.830470409739 검색시작 [ 3320 / 3359 ]
36.31338549737559,127.38135172273269 검색시작 [ 3321 / 3359 ]
36.3253088182674,127.37999543912699 검색시작 [ 3322 / 3359 ]
37.4817348774259,126.93055274785101 검색시작 [ 3323 / 3359 ]
37.459374663614895,126.950583978884 검색시작 [ 33

응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응

응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응?
응

,지역,일자,지명,주소,좌표,시,구,동
0,서울,3/3,피자몰회기역점,서울동대문구휘경동320-6,"37.5900925507337,127.05638644492198",서울특별시,동대문구,휘경1동
1,서울,3/3,코로나19경희대학교병원선별진료소,서울동대문구회기동1-5,"37.5938752004487,127.051832862704",서울특별시,동대문구,회기동
2,안동,2/24,잉크가이안동점,경북안동시태화동267-20,"36.56419459168093,128.71379228743163",경상북도,안동시,태화동
3,안동,2/28,대구지방법원,대구수성구범어동458-2,"35.862332285576194,128.62864828163129",대구광역시,수성구,범어2동
4,안동,2/20,파리바게뜨안동정하점,경북안동시정하동280-7,"36.55052044916591,128.7349767112714",경상북도,안동시,강남동
...,...,...,...,...,...,...,...,...
1787,경기,6/24,예송피아노&미술,경기구리시인창동527-15,"37.60627569836391,127.133219721922",경기도,구리시,인창동
1788,경기,6/25,예송피아노&미술,경기구리시인창동527-15,"37.60627569836391,127.133219721922",경기도,구리시,인창동
1789,경기,6/26,예송피아노&미술,경기구리시인창동527-15,"37.60627569836391,127.133219721922",경기도,구리시,인창동
1790,경기,6/22,코로나19만안구보건소선별진료소,경기안양시만안구안양동532-2,"37.385882567044696,126.93294929443",경기도,안양시 만안구,안양6동
